In [132]:
import pandas as pd
from datetime import datetime 
from datetime import timedelta
import numpy as np
import pickle

In [141]:
# Simulation
current_cell = "22:113"
current_datetime = pd.to_datetime("2015-06-01 14:00:00")

In [144]:
current_datetime.weekday()

0

In [157]:
time_tuple = extract_time(current_datetime)

In [177]:
manhattan_zones = {"27:80":"TriBeCa/Civic Center"}

In [178]:
poss = [(time_tuple + (zone, cell,)) for cell, zone in manhattan_zones.items()]

In [184]:
poss

[('Summer',
  'Jun',
  23,
  1,
  'Mon',
  False,
  14,
  'Daytime',
  0,
  'TriBeCa/Civic Center',
  '27:80')]

In [169]:
with open('../players/model/mnb.pkl', 'rb') as handle:
    mod = pickle.load(handle)
with open('../players/model/ohe.pkl', 'rb') as handle:
    ohe = pickle.load(handle)

In [171]:
print(ohe)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)


In [181]:
pred = ohe.transform(poss)

In [183]:
pred

<1x564 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [182]:
mod.predict(pred)

array([1.])

In [156]:
# def extract_time(dt):

#     # Extract the following predictors
#     # 'Season', 'Pickup_month', 'Pickup_week', 'Pickup_day', 'Pickup_wday',
#     # 'Weekend', 'Pickup_hour', 'Time', 'Pickup_minute', 'Zone',
#     # 'Pickup_cell'

#     seasons = {1: "Spring", 2: "Spring", 3: "Spring",
#                4: "Summer", 5: "Summer", 6: "Summer",
#                7: "Autumn", 8: "Autumn", 9: "Autumn",
#                10: "Winter", 11: "Winter", 12: "Winter"}

#     season = seasons.get(dt.month)
#     month = dt.strftime("%b")
#     week = dt.week
#     day = dt.day
#     weekday = dt.strftime("%a")
#     weekend = dt.weekday() >= 5
#     hour = dt.hour
#     time = "Daytime" if 6 <= dt.hour <= 18 else "Nighttime"
#     minute = dt.minute

#     return (season, month, week, day, weekday, weekend, hour, time, minute)

def extract_time(dt):

    # Extract the following predictors
    # 'Season', 'Pickup_month', 'Pickup_week', 'Pickup_day', 'Pickup_wday',
    # 'Weekend', 'Pickup_hour', 'Time', 'Pickup_minute', 'Zone',
    # 'Pickup_cell'

    seasons = {1: "Spring", 2: "Spring", 3: "Spring",
               4: "Summer", 5: "Summer", 6: "Summer",
               7: "Autumn", 8: "Autumn", 9: "Autumn",
               10: "Winter", 11: "Winter", 12: "Winter"}

    season = seasons.get(dt.month)
    month = dt.strftime("%b")
    week = int(dt.strftime("%V"))
    day = dt.day
    weekday = dt.strftime("%a")
    weekend = dt.weekday() >= 5
    hour = dt.hour
    time = "Daytime" if 6 <= dt.hour <= 18 else "Nighttime"
    minute = dt.minute

    return (season, month, week, day, weekday, weekend, hour, time, minute)

### Load graph and compute cost for all cells

In [12]:
import bfs

In [13]:
graph = bfs.load_graph("../players/data/graph.pkl")

In [14]:
costs = bfs.bfs(graph, current_cell)

# Just a subset of the dictionary
sp_dict = dict(list(costs.items()))

### Preprocess input

In [15]:
cell_to_location = pd.read_csv("../players/data/cell_to_location.csv")

seasons = {1:"Spring",2:"Spring",3:"Spring",
               4:"Summer", 5:"Summer", 6:"Summer",
               7:"Autumn", 8:"Autumn", 9:"Autumn",
               10:"Winter", 11:"Winter", 12:"Winter"}

zone_dict = cell_to_location.set_index("Cell").to_dict()["Zone"]
borough_dict = cell_to_location.set_index("Cell").to_dict()["Borough"]

In [16]:
def preprocess_cell(cid, dt):

    return[dt.strftime("%b"), str(dt.day), str(dt.hour), str(dt.minute), seasons.get(dt.month), 
           str(dt.week), dt.strftime("%a"),
            False if dt.weekday() < 5 else True,
           "Daytime" if 6 <= dt.hour <= 18 else "Nighttime",
           borough_dict[cid],
           zone_dict[cid],
           cid]

In [9]:
pred2d = []
for k, v in sp_dict.items():
    arrival_datetime = (current_datetime + timedelta(minutes=v)).replace(second=0)
    pred2d.append(preprocess_cell(k, current_datetime))

KeyError: '23:110'

### Load Glm
Frequency ~ PickupCell,day_of_week,minute_of_day

In [11]:
import pickle

In [12]:
# Poisson Glm
with open('../players/model/.pkl', 'rb') as handle:
    pois = pickle.load(handle)

In [13]:
from datetime import datetime 
from datetime import timedelta

In [14]:
import numpy as np

In [15]:
# pois_dict = sp_dict.copy()
# for i, (k, v) in enumerate(sp_dict.items()):
#     arr = df.values[i]
#     try:
#         pois_dict[k] = pois.predict(arr)
#     except ValueError:
#         pois_dict[k] = 0

pois.predict(df_train)

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error converting data to categorical: observation with value '22:113' does not match any of the expected levels (expected: ['20:40', '20:41', ..., '32:52', '32:53'])
    Number_trips~Pickup_hour+                         Pickup_minute+Pickup_cell
                                                                    ^^^^^^^^^^^

In [184]:
score_dict = sp_dict.copy()
for (k1,v1),(k2,v2) in zip(lr_dict.items(), sp_dict.items()):
    try:
        score_dict[k1] = v1/v2
    except ZeroDivisionError:
        score_dict[k1] = v1/0.0000001

In [185]:
best_cell = max(score_dict, key=score_dict.get)

In [186]:
print(lr_dict[best_cell], sp_dict[best_cell], score_dict[best_cell] )

1.0052490234375 23 0.043706479279891304


### Load lr
Frequency ~ PickupCell,day_of_week,minute_of_day

In [10]:
import pickle

In [176]:
# Linear Regression
with open('../model/lr.pickle', 'rb') as handle:
    lr = pickle.load(handle)

In [177]:
with open('../model/ohe.pickle', 'rb') as handle:
    ohe = pickle.load(handle)

In [180]:
### Compute time when arrive at destination cell to get minute of day
pred2d = []
for k, v in sp_dict.items():
    arr = [k, input[1], (input[2] + timedelta(minutes=v)).time().replace(second=0).strftime("%H:%M:%S")]
    pred2d.append(arr)
pred2d = np.array(pred2d) 
df = pd.DataFrame(pred2d)

In [182]:
### Compute time when arrive at destination cell to get minute of day
lr_dict = sp_dict.copy()
for i, (k, v) in enumerate(sp_dict.items()):
#     arr = np.array([k, input[1], (input[2] + timedelta(minutes=v)).time().replace(second=0).strftime("%H:%M:%S")]).reshape(1,-1)
    arr = df.values[i].reshape(1,-1)
    try:
        pred = ohe.transform(arr).toarray()
        lr_dict[k] = lr.predict(pred)[0]
    except ValueError:
        lr_dict[k] = 0

In [184]:
score_dict = sp_dict.copy()
for (k1,v1),(k2,v2) in zip(lr_dict.items(), sp_dict.items()):
    try:
        score_dict[k1] = v1/v2
    except ZeroDivisionError:
        score_dict[k1] = v1/0.0000001

In [185]:
best_cell = max(score_dict, key=score_dict.get)

In [186]:
print(lr_dict[best_cell], sp_dict[best_cell], score_dict[best_cell] )

1.0052490234375 23 0.043706479279891304
